In [34]:
#Aqui subire las consultas que hice en la base de datos en BigQuery sobre test hechos a nivel nacional en EE.UU con sus respectivos positivos y negativos, hospitalizaciones, gente con ventilador, muertes, entre otros.

from google.cloud import bigquery

import pandas as pd

# Create a "Client" object
client = bigquery.Client()

dataset_ref = client.dataset("covid19_jhu_csse", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

In [14]:
# Aqui se pueden ver los nonmbres de las tablas de la base de datos "covid19_jhu_csse"
tables = list(client.list_tables(dataset))

# Aqui se da la orden de imprimir los nombres de las tablas
for table in tables:  
    print(table.table_id)

confirmed_cases
deaths
recovered_cases
summary


In [15]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("deaths")

# API request - fetch the table
table = client.get_table(table_ref)

In [16]:
table.schema

[SchemaField('province_or_state', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('country_or_region', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('latitude', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('longitude', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('location_geom', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_22_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_23_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_24_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_25_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_26_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_27_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_28_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_29_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_30_20', 'STRING', 'NULLABLE', None, '', (), None),
 SchemaField('_1_31_20',

In [17]:
import db_dtypes

client.list_rows(table, max_results=5).to_dataframe()


,province_or_state,country_or_region,latitude,longitude,location_geom,_1_22_20,_1_23_20,_1_24_20,_1_25_20,_1_26_20,...,_2_28_23,_3_1_23,_3_2_23,_3_3_23,_3_4_23,_3_5_23,_3_6_23,_3_7_23,_3_8_23,_3_9_23
0,None,Antarctica,-71.9499,23.347,POINT(-71.9499 23.347),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Northern Territory,Australia,-12.4634,130.8456,POINT(-12.4634 130.8456),0,0,0,0,0,...,90,90,90,90,90,90,90,90,90,91
2,Grand Princess,Canada,0.0,0.0,POINT(0.0 0.0),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Northwest Territories,Canada,64.8255,-124.8457,POINT(64.8255 -124.8457),0,0,0,0,0,...,22,22,22,22,22,22,22,22,22,22
4,Prince Edward Island,Canada,46.5107,-63.4168,POINT(46.5107 -63.4168),0,0,0,0,0,...,92,92,92,92,92,92,92,93,93,93


In [23]:
#Ahora ya empezaremos a hacer las consultas sql
query = """
        SELECT *
        FROM `bigquery-public-data.covid19_jhu_csse.deaths`
        WHERE country_or_region = 'Canada'
        """

In [24]:
query_job = client.query(query)

Can_Prov = query_job.to_dataframe()

In [51]:
muertoscanada_df = Can_Prov.iloc[:,5:].transpose().reset_index()



In [52]:
muertoscanada_df

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,_1_22_20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,_1_23_20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,_1_24_20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,_1_25_20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,_1_26_20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,_3_5_23,0,22,92,0,32,7,1,1882,5602,5218,825,16186,311,18131,786,2460
1139,_3_6_23,0,22,92,0,32,7,1,1890,5602,5218,834,16186,318,18132,794,2464
1140,_3_7_23,0,22,93,0,32,7,1,1890,5602,5218,834,16186,318,18144,794,2464
1141,_3_8_23,0,22,93,0,32,7,1,1890,5622,5218,834,16186,318,18153,794,2464


In [53]:
muertoscanada_df["index"]=pd.to_datetime(muertoscanada_df["index"],format='_%m_%d_%y')

In [43]:
muertoscanada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   1143 non-null   datetime64[ns]
 1   0       1143 non-null   object        
 2   1       1143 non-null   object        
 3   2       1143 non-null   object        
 4   3       1143 non-null   object        
 5   4       1143 non-null   object        
 6   5       1143 non-null   object        
 7   6       1143 non-null   object        
 8   7       1143 non-null   object        
 9   8       1143 non-null   object        
 10  9       1143 non-null   object        
 11  10      1143 non-null   object        
 12  11      1143 non-null   object        
 13  12      1143 non-null   object        
 14  13      1143 non-null   object        
 15  14      1143 non-null   object        
 16  15      1143 non-null   object        
 17  total   1143 non-null   object        
dtypes: datet

In [55]:
muertoscanada_df[[0 , 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]]=muertoscanada_df[[0 , 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]].astype(int)
muertoscanada_df['total'] = muertoscanada_df.iloc[:, 1:].sum(axis=1)

muertoscanada_df

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,total
0,2020-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,2023-03-05,0,22,92,0,32,7,1,1882,5602,5218,825,16186,311,18131,786,2460,51555
1139,2023-03-06,0,22,92,0,32,7,1,1890,5602,5218,834,16186,318,18132,794,2464,51592
1140,2023-03-07,0,22,93,0,32,7,1,1890,5602,5218,834,16186,318,18144,794,2464,51605
1141,2023-03-08,0,22,93,0,32,7,1,1890,5622,5218,834,16186,318,18153,794,2464,51634


In [48]:
muertoscanada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   1143 non-null   datetime64[ns]
 1   0       1143 non-null   int32         
 2   1       1143 non-null   int32         
 3   2       1143 non-null   object        
 4   3       1143 non-null   object        
 5   4       1143 non-null   object        
 6   5       1143 non-null   object        
 7   6       1143 non-null   object        
 8   7       1143 non-null   object        
 9   8       1143 non-null   object        
 10  9       1143 non-null   object        
 11  10      1143 non-null   object        
 12  11      1143 non-null   object        
 13  12      1143 non-null   object        
 14  13      1143 non-null   object        
 15  14      1143 non-null   object        
 16  15      1143 non-null   object        
 17  total   1143 non-null   object        
dtypes: datet

In [57]:
muertoscanada_df_final=muertoscanada_df[["index","total"]]
muertoscanada_df_final

,index,total
0,2020-01-22,0
1,2020-01-23,0
2,2020-01-24,0
3,2020-01-25,0
4,2020-01-26,0
...,...,...
1138,2023-03-05,51555
1139,2023-03-06,51592
1140,2023-03-07,51605
1141,2023-03-08,51634
